In [1]:
#import necessary libraries
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import keras


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM, SimpleRNN

In [3]:
import numpy as np
a = open("data_shuffle_input.csv")
net_data_input = np.genfromtxt(a, delimiter=",")
b = open("data_shuffle_target.csv")
net_data_target = np.genfromtxt(b, delimiter=",")
c = open("mydata_shuffle_input.csv")
net_mydata_input = np.genfromtxt(c, delimiter=",")
d = open("mydata_shuffle_target.csv")
net_mydata_target = np.genfromtxt(d, delimiter=",")

In [4]:
import random
from sklearn.model_selection import cross_val_score

net_data_input = net_data_input.reshape(net_data_input.shape[0],1,784)
net_mydata_input = net_mydata_input.reshape(net_mydata_input.shape[0],1,784)
net_data_input = net_data_input / 255

In [5]:
acc_per_fold = []
loss_per_fold = []


In [6]:
num = []
for i in range(len(net_data_target)):
    if net_data_target[i][0] == 1:
        num.append(0)
    if net_data_target[i][1] == 1:
        num.append(1)
    if net_data_target[i][2] == 1:
        num.append(2)
    if net_data_target[i][3] == 1:
        num.append(3)
    if net_data_target[i][4] == 1:
        num.append(4)
    if net_data_target[i][5] == 1:
        num.append(5)
    if net_data_target[i][6] == 1:
        num.append(6)
    if net_data_target[i][7] == 1:
        num.append(7)
    if net_data_target[i][8] == 1:
        num.append(8)
    if net_data_target[i][9] == 1:
        num.append(9)
num = np.asarray(num)        
print(type(num))


<class 'numpy.ndarray'>


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5)
acc_per_fold = []

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(net_data_input, net_data_target, net_mydata_input):

  # Define the model architecture for net data
    rnn_model_data = Sequential()
    #rnn_model.add(Flatten())
    rnn_model_data.add(LSTM(256,input_shape=net_data_input[0].shape,activation="tanh"))
    #rnn_model.add(SimpleRNN(128,input_shape=xtrain[0].shape,return_sequences=True))
    #rnn_model.add(SimpleRNN(128))
    rnn_model_data.add(Dense(units=10,activation="softmax"))

  # Compile the model
    rnn_model_data.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} data')

  # Fit data to model
    history_data = rnn_model_data.fit(net_data_input[train], net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    predictions_data_train = rnn_model_data.predict(net_data_input[train])
    classes_data_train = np.argmax(predictions_data_train, axis = 1)
    classes_data_train = tf.keras.utils.to_categorical(classes_data_train,10)
    
    predictions_data_test = rnn_model_data.predict(net_data_input[test])
    classes_data_test = np.argmax(predictions_data_test, axis = 1)
    classes_data_test = tf.keras.utils.to_categorical(classes_data_test,10)
        
  # Define the model architecture for net mydata
    rnn_model_mydata = Sequential()
    #rnn_model.add(Flatten())
    rnn_model_mydata.add(LSTM(256,input_shape=net_mydata_input[0].shape,activation="tanh"))
    #rnn_model.add(SimpleRNN(128,input_shape=xtrain[0].shape,return_sequences=True))
    #rnn_model.add(SimpleRNN(128))
    rnn_model_mydata.add(Dense(units=10,activation="softmax"))

  # Compile the model
    rnn_model_mydata.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} mydata')
  # Fit data to model
    history = rnn_model_mydata.fit(net_mydata_input[train], net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    

    
    predictions_mydata_train = rnn_model_mydata.predict(net_mydata_input[train])
    classes_mydata_train = np.argmax(predictions_mydata_train, axis = 1)
    classes_mydata_train = tf.keras.utils.to_categorical(classes_mydata_train,10)

    predictions_mydata_test = rnn_model_mydata.predict(net_mydata_input[test])
    classes_mydata_test = np.argmax(predictions_mydata_test, axis = 1)
    classes_mydata_test = tf.keras.utils.to_categorical(classes_mydata_test,10)

    
    #make train list input
    joined_list_train = []
    joined_list_test = []

    for i in range(800):
        b = [*classes_data_train[i], *classes_mydata_train[i]]
        joined_list_train.append(b)
        
    for j in range(200):
        a = [*classes_data_test[j], *classes_mydata_test[j]]
        joined_list_test.append(a)
    
    
    joined_list_train = np.asarray(joined_list_train)
    joined_list_test = np.asarray(joined_list_test)
    
    joined_list_train = joined_list_train.reshape(joined_list_train.shape[0],1,20)

    joined_list_test = joined_list_test.reshape(joined_list_test.shape[0],1,20)
    
  # Define the model architecture for net data
    print(joined_list_train[0].shape)

    rnn_model_outputs = Sequential()
    #rnn_model.add(Flatten())
    rnn_model_outputs.add(LSTM(256,input_shape=joined_list_train[0].shape,activation="tanh"))
    #rnn_model.add(SimpleRNN(128,input_shape=xtrain[0].shape,return_sequences=True))
    #rnn_model.add(SimpleRNN(128))
    rnn_model_outputs.add(Dense(units=10,activation="softmax"))

  # Compile the model
    rnn_model_outputs.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])


  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} outputs')

  # Fit data to model
    history = rnn_model_outputs.fit(joined_list_train, net_data_target[train],
              batch_size=128,
              epochs=50,
              verbose=1)
    scores = rnn_model_outputs.evaluate(joined_list_test, net_data_target[test], verbose=0)
    #print(f'Score for fold {fold_no}: {rnn_model.metrics_names[0]} of {scores[0]}; {rnn_model.metrics_names[1]} of {scores[1]*100}%')
    #acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    predictions = rnn_model_outputs.predict(joined_list_test)
    classes = np.argmax(predictions, axis = 1)
    print(num[test])
    acc_per_fold.append(accuracy_score(num[test],classes))
    
    print('accuracy is ' + str(accuracy_score(num[test],classes)))
    
  # Generate generalization metrics
    
    

  # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 data
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 1s 1ms/sample - loss: 2.0676 - acc: 0.4462
Epoch 2/50
800/800 [==============================] - 0s 230us/sample - loss: 1.4372 - acc: 0.7613
Epoch 3/50
800/800 [==============================] - 0s 213us/sample - loss: 0.9041 - acc: 0.8200
Epoch 4/50
800/800 [==============================] - 0s 184us/sample - loss: 0.6175 - acc: 0.8600
Epoch 5/50
800/800 [==============================] - 0s 180us/sample - loss: 0.4607 - acc: 0.8875
Epoch 6/50
800/800 [==============================] - 0s 172us/sample - loss: 0.3727 - acc: 0.9087
Epoch 7/50
800/800 [==============================] - 0s 170us/sample - loss: 0.3102 - acc: 0.9237
Epoch 8/50
800/800 [==============================] - 0s 183us/sample - loss: 0.2643 - acc: 0.9375
Epoch 9/50
800/800 [==============================] - 0s 178us/sample - loss: 0.2292 - acc:

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


------------------------------------------------------------------------
Training for fold 1 mydata
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 2s 2ms/sample - loss: 2.1686 - acc: 0.2512
Epoch 2/50
800/800 [==============================] - 0s 284us/sample - loss: 1.0584 - acc: 0.9038
Epoch 3/50
800/800 [==============================] - 0s 284us/sample - loss: 0.5619 - acc: 0.9737
Epoch 4/50
800/800 [==============================] - 0s 300us/sample - loss: 0.3017 - acc: 0.9962
Epoch 5/50
800/800 [==============================] - 0s 279us/sample - loss: 0.1701 - acc: 1.0000
Epoch 6/50
800/800 [==============================] - 0s 272us/sample - loss: 0.1013 - acc: 1.0000
Epoch 7/50
800/800 [==============================] - 0s 297us/sample - loss: 0.0666 - acc: 1.0000
Epoch 8/50
800/800 [==============================] - 0s 318us/sample - loss: 0.0475 - acc: 1.0000
Epoch 9/50
800/800 [==============================] - 0s 305us/sample - loss: 0.0359 - ac

800/800 [==============================] - 0s 104us/sample - loss: 0.0232 - acc: 1.0000
Epoch 31/50
800/800 [==============================] - 0s 98us/sample - loss: 0.0213 - acc: 1.0000
Epoch 32/50
800/800 [==============================] - 0s 120us/sample - loss: 0.0196 - acc: 1.0000
Epoch 33/50
800/800 [==============================] - 0s 103us/sample - loss: 0.0181 - acc: 1.0000
Epoch 34/50
800/800 [==============================] - 0s 102us/sample - loss: 0.0168 - acc: 1.0000
Epoch 35/50
800/800 [==============================] - 0s 123us/sample - loss: 0.0157 - acc: 1.0000
Epoch 36/50
800/800 [==============================] - 0s 98us/sample - loss: 0.0146 - acc: 1.0000
Epoch 37/50
800/800 [==============================] - 0s 104us/sample - loss: 0.0137 - acc: 1.0000
Epoch 38/50
800/800 [==============================] - 0s 101us/sample - loss: 0.0129 - acc: 1.0000
Epoch 39/50
800/800 [==============================] - 0s 103us/sample - loss: 0.0121 - acc: 1.0000
Epoch 40/50
80

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


...

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


------------------------------------------------------------------------
Training for fold 5 mydata
Train on 800 samples
Epoch 1/50
800/800 [==============================] - 0s 528us/sample - loss: 2.1404 - acc: 0.2587
Epoch 2/50
800/800 [==============================] - 0s 121us/sample - loss: 1.0358 - acc: 0.8950
Epoch 3/50
800/800 [==============================] - 0s 112us/sample - loss: 0.5451 - acc: 0.9750
Epoch 4/50
800/800 [==============================] - 0s 159us/sample - loss: 0.2984 - acc: 0.9912
Epoch 5/50
800/800 [==============================] - 0s 122us/sample - loss: 0.1702 - acc: 0.9975
Epoch 6/50
800/800 [==============================] - 0s 126us/sample - loss: 0.1050 - acc: 1.0000
Epoch 7/50
800/800 [==============================] - 0s 141us/sample - loss: 0.0695 - acc: 1.0000
Epoch 8/50
800/800 [==============================] - 0s 119us/sample - loss: 0.0492 - acc: 1.0000
Epoch 9/50
800/800 [==============================] - 0s 113us/sample - loss: 0.0373 - 

800/800 [==============================] - 0s 63us/sample - loss: 0.0231 - acc: 1.0000
Epoch 31/50
800/800 [==============================] - 0s 70us/sample - loss: 0.0212 - acc: 1.0000
Epoch 32/50
800/800 [==============================] - 0s 84us/sample - loss: 0.0196 - acc: 1.0000
Epoch 33/50
800/800 [==============================] - 0s 78us/sample - loss: 0.0181 - acc: 1.0000
Epoch 34/50
800/800 [==============================] - 0s 67us/sample - loss: 0.0168 - acc: 1.0000
Epoch 35/50
800/800 [==============================] - 0s 63us/sample - loss: 0.0156 - acc: 1.0000
Epoch 36/50
800/800 [==============================] - 0s 73us/sample - loss: 0.0146 - acc: 1.0000
Epoch 37/50
800/800 [==============================] - 0s 69us/sample - loss: 0.0137 - acc: 1.0000
Epoch 38/50
800/800 [==============================] - 0s 67us/sample - loss: 0.0128 - acc: 1.0000
Epoch 39/50
800/800 [==============================] - 0s 69us/sample - loss: 0.0121 - acc: 1.0000
Epoch 40/50
800/800 [=

D:\anaconda\lib\site-packages\keras\engine\training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[6 6 6 1 8 0 6 9 9 2 0 3 6 7 1 3 3 5 8 0 5 7 2 3 9 0 5 7 2 5 2 3 8 5 8 0 8
 9 8 7 9 8 7 5 0 5 1 7 3 1 6 3 6 5 6 9 2 5 1 3 6 2 9 1 4 8 7 8 0 5 2 7 4 0
 2 9 0 2 2 7 5 8 8 2 2 9 6 9 4 2 0 0 5 7 3 9 1 8 5 1 1 9 0 4 2 0 1 4 0 1 2
 2 8 0 0 1 6 5 4 4 4 8 1 5 8 8 1 4 8 8 3 9 0 7 4 8 4 2 2 5 8 1 3 9 1 0 3 0
 8 3 4 9 2 4 0 6 8 0 2 9 9 5 7 1 7 2 3 5 7 5 1 8 5 0 1 2 1 5 1 4 8 4 0 2 9
 0 0 1 6 0 6 8 5 8 3 4 8 6 3 5]
accuracy is 0.82


In [8]:

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.48918826460838316 - Accuracy: 0.8%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.7333382403850556 - Accuracy: 0.8%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.6051040506362915 - Accuracy: 0.825%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.4647685217857361 - Accuracy: 0.8%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.5744698238372803 - Accuracy: 0.82%
------------------------------------------------------------------------
Average scores for all folds:
> Accuracy: 0.8089999999999999 (+- 0.01113552872566)
> Loss: 0.5733737802505494
------------------------------------------------------------------------
